# Routing by semantic similarity

In [10]:
import os
from dotenv import load_dotenv

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [11]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [12]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

In [13]:
embeddings = OpenAIEmbeddings()
prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


In [14]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

In [15]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI()
    | StrOutputParser()
)

In [16]:
print(chain.invoke("What's a black hole"))

Using PHYSICS
A black hole is a region in space where gravity is extremely strong, so strong that nothing, not even light, can escape its gravitational pull. It is formed when a massive star collapses under its own weight, causing its core to condense into a single point of infinite density called a singularity. The boundary of a black hole is called the event horizon, which marks the point of no return. Anything that crosses the event horizon is trapped inside the black hole's intense gravitational field. Black holes are fascinating objects that continue to captivate scientists and astronomers, and there is still much to learn about them.


In [17]:
print(chain.invoke("What's a path integral"))

Using MATH
Thank you for the kind words! I'll be happy to help you with your question.

A path integral, also known as a functional integral, is a concept in mathematics and theoretical physics. It is used to calculate the probability amplitudes or expectation values of physical quantities in quantum mechanics and quantum field theory.

In simple terms, a path integral involves summing over all possible paths or trajectories that a particle or field can take between two points in spacetime. Each path is assigned a certain weight or probability amplitude, and the total probability amplitude is obtained by summing over all these contributions.

The concept of a path integral was first introduced by Richard Feynman as a way to formulate quantum mechanics in terms of the principle of least action. It allows us to describe the behavior of particles or fields in a probabilistic manner, taking into account all possible paths they could follow.

To calculate a path integral, one typically brea